In [1]:
import pandas as pd

In [3]:
#Reading the processed dataset
cic_df=pd.read_parquet("processed_dataset.parquet")

In [4]:
#Reading the first 5 rows
cic_df.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Fwd_Packets_Length_Total,Bwd_Packets_Length_Total,Fwd_Packet_Length_Max,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Mean,...,Avg_Bwd_Segment_Size,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Fwd_Win_Bytes,Fwd_Act_Data_Packets,Fwd_Seg_Size_Min,isMalicious,attack_id
5968290,3813760.0,5.0,3.0,935.0,397.0,935.0,187.000,418.144714,397.0,132.333328,...,132.333328,5.0,935.0,3.0,397.0,219.0,1.0,32.0,0,0
8285216,396839.0,2.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,...,0.000000,2.0,0.0,0.0,0.0,63326.0,0.0,20.0,0,0
8349977,1914354.0,8.0,7.0,1144.0,1581.0,677.0,143.000,227.969925,1173.0,225.857147,...,225.857147,8.0,1144.0,7.0,1581.0,8192.0,5.0,20.0,0,0
7180832,4002.0,6.0,0.0,2064.0,0.0,440.0,44.000,148.722565,0.0,0.000000,...,0.000000,6.0,2064.0,0.0,0.0,8192.0,5.0,8.0,1,3
2324438,5368715.0,8.0,6.0,355.0,232.0,198.0,44.375,75.864426,1460.0,108.000000,...,108.000000,8.0,355.0,6.0,232.0,8192.0,3.0,20.0,0,0


In [9]:
#Fetching distribution of different attack ids in the dataset
cic_df['attack_id'].value_counts()

attack_id
0    1437467
3     246982
1      29348
2      20546
Name: count, dtype: int64

Due to imbalanced nature of the dataset, we will perform Undersampling. 

Advantages: -
1. This will help us avoid creating duplication.
2. It will ensure all records have equal number of coounts, thus, reducing bias.

Disadvantages: -
1. It will lead to loss of data from the training dataset.

In [10]:
attack_id_counts=cic_df['attack_id'].value_counts()
target_count=attack_id_counts.min()
print("Minimum count of attack_id: ", target_count)

Minimum count of attack_id:  20546


In [11]:
#Performing undersampling on the dataset
undersampled_cic_df=pd.concat([
    cic_df[cic_df['attack_id']==0].sample(target_count, replace=False),
    cic_df[cic_df['attack_id']==1].sample(target_count, replace=False),
    cic_df[cic_df['attack_id']==2],
    cic_df[cic_df['attack_id']==3].sample(target_count, replace=False)], axis=0)

#Shuffling the dataset after undersampling
undersampled_cic_df=undersampled_cic_df.sample(frac=1).reset_index(drop=True)

In [12]:
undersampled_cic_df.shape

(82184, 47)

In [13]:
undersampled_cic_df.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Fwd_Packets_Length_Total,Bwd_Packets_Length_Total,Fwd_Packet_Length_Max,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Mean,...,Avg_Bwd_Segment_Size,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Fwd_Win_Bytes,Fwd_Act_Data_Packets,Fwd_Seg_Size_Min,isMalicious,attack_id
0,2999994.0,4.0,0.0,2064.0,0.0,516.0,44.000000,0.000000,0.0,0.00,...,0.00,4.0,2064.0,0.0,0.0,8192.0,3.0,20.0,1,3
1,11487.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1,1
2,9818.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1,1
3,10138.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1,1
4,109.0,3.0,1.0,53.0,0.0,53.0,17.666666,30.599564,0.0,0.00,...,0.00,3.0,53.0,1.0,0.0,1080.0,1.0,20.0,0,0


In [14]:
binary_cic_df=undersampled_cic_df.drop('attack_id', axis=1)
multiclass_cic_df=undersampled_cic_df.drop('isMalicious', axis=1)

In [15]:
binary_cic_df.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Fwd_Packets_Length_Total,Bwd_Packets_Length_Total,Fwd_Packet_Length_Max,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Mean,...,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Fwd_Win_Bytes,Fwd_Act_Data_Packets,Fwd_Seg_Size_Min,isMalicious
0,2999994.0,4.0,0.0,2064.0,0.0,516.0,44.000000,0.000000,0.0,0.00,...,44.000000,0.00,4.0,2064.0,0.0,0.0,8192.0,3.0,20.0,1
1,11487.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
2,9818.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
3,10138.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
4,109.0,3.0,1.0,53.0,0.0,53.0,17.666666,30.599564,0.0,0.00,...,17.666666,0.00,3.0,53.0,1.0,0.0,1080.0,1.0,20.0,0


In [16]:
multiclass_cic_df.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Fwd_Packets_Length_Total,Bwd_Packets_Length_Total,Fwd_Packet_Length_Max,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Mean,...,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Fwd_Win_Bytes,Fwd_Act_Data_Packets,Fwd_Seg_Size_Min,attack_id
0,2999994.0,4.0,0.0,2064.0,0.0,516.0,44.000000,0.000000,0.0,0.00,...,44.000000,0.00,4.0,2064.0,0.0,0.0,8192.0,3.0,20.0,3
1,11487.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
2,9818.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
3,10138.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
4,109.0,3.0,1.0,53.0,0.0,53.0,17.666666,30.599564,0.0,0.00,...,17.666666,0.00,3.0,53.0,1.0,0.0,1080.0,1.0,20.0,0


In [17]:
binary_cic_df.shape

(82184, 46)

In [19]:
multiclass_cic_df.shape

(82184, 46)

Writing the balanaced datasets into new files: -
1. For Binary classification: - binary_training_data.parquet
2. For Multi-class classification: - multiclass_training_data.parquet

In [20]:
binary_cic_df.to_parquet('binary_training_data.parquet')
multiclass_cic_df.to_parquet('multiclass_training_data.parquet')